In [5]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib
import tkinter as tk
import numpy as np
from tkinter import messagebox
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg,  
    NavigationToolbar2Tk) 
from matplotlib.backend_bases import MouseEvent
from matplotlib.figure import Figure

# Plot function to plot the filtered data and check the bad shots according to contractual limits
def plot():
    
    final.to_sql('ac_log1', engine, if_exists='replace', index=False)
    final.to_csv(output, index=False)   
    matplotlib.use('TkAgg')
    

    # Source Volume, Pressure, Separation and Depth metioned in contracts    
    choiceSourceV=3400
    choicePressureV=2000
    choiceSeparation=6
    choiceDepth=5
    
    # Limits flexibility allowed in contract lmits 
    Volume_Limit_low=str(int(0.9*int(choiceSourceV)))
    Volume_Limit_high=str(int(1.1*int(choiceSourceV)))
    Pressure_Limit_low=str(int(0.9*int(choicePressureV)))
    Pressure_Limit_high=str(int(1.1*int(choicePressureV)))
    Sep_Limit_low=str(int(0.8*int(choiceSeparation)))
    Sep_Limit_high=str(int(1.2*int(choiceSeparation)))
    Depth_Limit_low=str(int(0.8*int(choiceDepth)))
    Depth_Limit_high=str(int(1.2*int(choiceDepth)))

    def callback(event):
        print ("clicked at", event.x, event.y)

    root = tk.Tk()
    f = Figure(figsize=(5,4), dpi=100)
    a = f.add_subplot(411)
    a.set_ylabel('Volume')
    V_results= engine.execute("Select Station, Volume  from ac_log1 where Volume <= " + Volume_Limit_low +" OR Volume >= " + Volume_Limit_high)
    
    station=[];
    station_record=[];
    Pressure=[];
    Volume=[];
    Separation=[];
    GunDepth=[];
    
    for row in V_results.fetchall():
        station.append(row[0])
        station_record.append(row[0])
        Volume.append(row[1])   
        
    Vol_low_limit= int(Volume_Limit_low)*np.ones(len(final['Shot']))
    Vol_high_limit= int(Volume_Limit_high)*np.ones(len(final['Shot']))
    
    a.plot(final['Station'],final['Volume'], color='green',linestyle='',marker='o')
    a.plot(station,Volume,color='red', linestyle = '', marker='o')
    a.plot(final['Station'],Vol_low_limit, '--b')
    a.plot(final['Station'],Vol_high_limit, '--b')

    P_results= engine.execute("Select Station, Pressure  from ac_log1 where Pressure <= " + Pressure_Limit_low + " OR Pressure >= " + Pressure_Limit_high)
    station=[];
    
    for row in P_results.fetchall():
        station.append(row[0])
        station_record.append(row[0])
        Pressure.append(row[1])

    a = f.add_subplot(412)
    a.set_ylabel('Pressure')
    a.plot(final['Station'], final['Pressure'], color='green',linestyle='',marker='o')
    a.plot(station,Pressure, color='red',linestyle = '', marker='o')
    Pres_low_limit= int(Pressure_Limit_low)*np.ones(len(final['Station']))
    Pres_high_limit= int(Pressure_Limit_high)*np.ones(len(final['Station']))
    a.plot(final['Station'],Pres_low_limit, '--b')
    a.plot(final['Station'],Pres_high_limit, '--b')

    Sep_results= engine.execute("Select Station, GSeparation  from ac_log1 where GSeparation <= " + Sep_Limit_low + " OR GSeparation >= " + Sep_Limit_high)
    station=[];
    
    for row in Sep_results.fetchall():
        station.append(row[0])
        station_record.append(row[0])
        Separation.append(row[1])

    a = f.add_subplot(413)
    a.set_ylabel('GSeparation')
    a.plot(final['Station'],final['GSeparation'], color='green',linestyle='',marker='o')
    Sep_low_limit= int(Sep_Limit_low)*np.ones(len(final['Station']))
    Sep_high_limit= int(Sep_Limit_high)*np.ones(len(final['Station']))
    a.plot(final['Station'],Sep_low_limit, '--b')
    a.plot(final['Station'],Sep_high_limit, '--b')
    
    if station:
        a.plot(station,Separation, color='red',linestyle = '',marker='o')

    

    a = f.add_subplot(414)
    a.set_ylabel('GunDepth')
    a.set_xlabel('SHOT Number')
    
    Depth_results= engine.execute("Select Station, GunDepth from ac_log1 where GunDepth <= " + Depth_Limit_low + " OR GunDepth >= " + Depth_Limit_high)
    station=[];
    
    for row in Depth_results.fetchall():
        station.append(row[0])
        station_record.append(row[0])
        GunDepth.append(row[1])
        
    a.plot(final['Station'], final['GunDepth'], color='green',linestyle='',marker='o')
    a.plot(station,GunDepth, color='red', linestyle = '', marker='o')
    
    Depth_low_limit= int(Depth_Limit_low)*np.ones(len(final['Station']))
    Depth_high_limit= int(Depth_Limit_high)*np.ones(len(final['Station']))
    a.plot(final['Station'],Depth_low_limit, '--b')
    a.plot(final['Station'],Depth_high_limit, '--b')

    canvas = FigureCanvasTkAgg(f, master=root)
    canvas.draw()
    canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

    canvas.mpl_connect('button_press_event',callback)

    def callback(event):
        print ("clicked at", event.xdata, event.ydata)

    toolbar = NavigationToolbar2Tk( canvas, root )
    toolbar.update()
    canvas._tkcanvas.pack(side=tk.TOP, fill=tk.BOTH, expand=1)
    df2 = pd. DataFrame(station_record) 
    df2.to_csv(bad_shots, index=False)

   
    
    station_number=np.unique(station_record)
    num=len(station_number)  
    
    len(station_record)
    try:
        bad_shot_percentage=round((int(len(station_number)*100)/len(final['Shot'])),2)
    except ZeroDivisionError:
        bad_shot_percentage=0

    msg = "Total shots-"+str(len(final['Station']))+"\n"+"Percentage of bad shots "+ str(bad_shot_percentage) +"%"  +"\n" + "No. of Bad shots due to Volume variance-" + str(len(Volume))+"\n" + "No. of Bad shots due to Pressure variance-" + str(len(Pressure))+ "\n" + "No. of Bad shots due to Separation variance-" + str(len(Separation)) +"\n" + "No. of Bad shots due to GunDepth variance-" + str(len(GunDepth))
    messagebox.showinfo("Number of bad shots", msg)

    toolbar
    root.mainloop()

In [8]:

# tkinter is  a library to make gui, all the graphical interface features are imported by tkinther
#sqlite3 library is used to import sql
#pandas to import excel file(database) in form of dataframe
# sqlalchemy for communication between database and sql

import tkinter as tk
from tkinter import *
from tkinter import filedialog
import sqlite3
import pandas as pd
from sqlalchemy import create_engine, MetaData
import time

#output variable to save our filtered data in .csv format
global output 
output= 'output.csv'
global bad_shots 
bad_shots= 'bad_shots.csv'



# -------sorting function to get filter data according to defined limits on chosen variable

def sorting(choice1, start_point, end_point):
    #result & final variable to save our filtered data on temporary basis for each filter selection
    global results
    global final 
    results= engine.execute("Select * from ac_log2 Where " + choice1 +" >= " + start_point + " AND " + choice1 +" <= " + end_point)  
    final=pd.DataFrame(results, columns=df.columns)
    

#------Functions of Source QC GUI      
# ----Functions to define the funcion of buttons
#-------Please check the line first from where the function myClick0() is called

def myClick0():
     
    
    def ok1(): 
        choice1=variable1.get()
        e3 = tk.Entry(root)        
        e3.insert(END, 'NA')
        e3.grid(row=2, column=3)
        e4 = tk.Entry(root) 
        e4.insert(END, 'NA')
        e4.grid(row=2, column=6)
        
       
        def start():
            global start_point 
            start_point=e3.get()

        def end():
            global end_point
            end_point=e4.get()
            
           
            sorting(choice1,start_point,end_point)    
            final.to_sql('ac_log2', engine, if_exists='replace', index=False)
            
        my_string_var5.set("Start- "+choice1)

        my_label = tk.Label(root,
                         textvariable = my_string_var5).grid(row=2, column=2)

        my_string_var6.set("End- "+choice1)
        my_label = tk.Label(root,
                     textvariable = my_string_var6).grid(row=2, column=5)

        btn_1_1 = tk.Button(root,
                   text = "Enter",
                   command = start)
        btn_1_2 = tk.Button(root,
                   text = "Enter",
                   command = end)

        btn_1_1.grid(row=2,column=4)
        btn_1_2.grid(row=2,column=7)
        
        

    def ok2(): 
        
        choice2=variable2.get()
        e3 = tk.Entry(root)
        e3.insert(END, 'NA')
        e3.grid(row=3, column=3)
        e4 = tk.Entry(root) 
        e4.insert(END, 'NA')
        e4.grid(row=3, column=6)

    
        def start():
            global start_point 
            start_point=e3.get()
        

        def end():
            global end_point
            end_point=e4.get()
            
            sorting(choice2,start_point,end_point)
             
            
        my_string_var7.set("Start- "+choice2)

        my_label = tk.Label(root,
                         textvariable = my_string_var7).grid(row=3, column=2)

        my_string_var8.set("End- "+choice2)
        my_label = tk.Label(root,
                     textvariable = my_string_var8).grid(row=3, column=5)

        btn_2_1 = tk.Button(root,
                   text = "Enter",
                   command = start)
        btn_2_2 = tk.Button(root,
                   text = "Enter",
                   command = end)

        btn_2_1.grid(row=3,column=4)
        btn_2_2.grid(row=3,column=7)
        
       
        

    def ok3(): 
        final.to_sql('ac_log2', engine, if_exists='replace', index=False)
        choice3=variable3.get()
        e3 = tk.Entry(root)
        e3.insert(END, 'NA')
        e3.grid(row=4, column=3)
        e4 = tk.Entry(root) 
        e4.insert(END, 'NA')
        e4.grid(row=4, column=6)
        
        
        def start():
            global start_point 
            start_point=e3.get()
    

        def end():
            global end_point
            end_point=e4.get()
            
            sorting(choice3,start_point,end_point)

        my_string_var9.set("Start- "+choice3)

        my_label = tk.Label(root,
                         textvariable = my_string_var9).grid(row=4, column=2)

        my_string_var10.set("End- "+choice3)
        my_label = tk.Label(root,
                     textvariable = my_string_var10).grid(row=4, column=5)

        btn_2_1 = tk.Button(root,
                   text = "Enter",
                   command = start)
        btn_2_2 = tk.Button(root,
                   text = "Enter",
                   command = end)

        btn_2_1.grid(row=4,column=4)
        btn_2_2.grid(row=4,column=7)
    
        
    
    def ok4(): 
        final.to_sql('ac_log2', engine, if_exists='replace', index=False)
        choice4=variable4.get()
        e3 = tk.Entry(root)
        e3.insert(END, 'NA')
        e3.grid(row=5, column=3)
        e4 = tk.Entry(root) 
        e4.insert(END, 'NA')
        e4.grid(row=5, column=6)

        if choice4 != E:
            def start():
                global start_point 
                start_point=e3.get()

            def end():
                global end_point
                end_point=e4.get()
                sorting(choice4,start_point,end_point)

            

            my_string_var11.set("Start- "+choice4)

            my_label = tk.Label(root,
                             textvariable = my_string_var11).grid(row=5, column=2)

            my_string_var12.set("End- "+choice4)
            my_label = tk.Label(root,
                         textvariable = my_string_var12).grid(row=5, column=5)

            btn_2_1 = tk.Button(root,
                       text = "Enter",
                       command = start)
            btn_2_2 = tk.Button(root,
                       text = "Enter",
                       command = end)

            btn_2_1.grid(row=5,column=4)
            btn_2_2.grid(row=5,column=7)
        
           
            
    
    global df
    df= pd.read_csv(filename);
    
    global engine    
#     start = time.time()
    engine = create_engine('sqlite://', echo=False)
    conn = engine.connect()
    metadata = MetaData(conn)
    df.to_sql('ac_log2', con=conn, if_exists='replace', index=False,chunksize=1000)
#     df.to_sql('ac_log2', engine, if_exists='replace', index=False, chunksize=1000)
#     stop = time.time() 
#     print('finished in %s seconds', stop)
    OPTIONS1 = [
    "JDay",
    "Station",
    "Line",
    "Vessel",
    "Sequence"
    ] 
    OPTIONS2=df.columns.values.tolist()

    btn_1_0 = tk.Button(root, text="OK", command=ok1)
    btn_2_0 = tk.Button(root, text="OK", command=ok2)
    btn_3_0 = tk.Button(root, text="OK", command=ok3)
    btn_4_0 = tk.Button(root, text="OK", command=ok4)


    variable1 = StringVar(root)
    variable2 = StringVar(root)
    variable3 = StringVar(root)
    variable4 = StringVar(root)
    my_string_var5 = StringVar()
    my_string_var6 = StringVar()
    my_string_var7 = StringVar()
    my_string_var8 = StringVar()
    my_string_var9 = StringVar()
    my_string_var10 = StringVar()
    my_string_var11 = StringVar()
    my_string_var12 = StringVar()
    
    variable1.set(OPTIONS1[0]) 
    variable2.set('Station')
    variable3.set('Line')
    variable4.set('Vessel')

    btn_1 = tk.OptionMenu(root, variable1, *OPTIONS1)
    btn_2 = tk.OptionMenu(root,
                  variable2, *OPTIONS1)
    choice2=variable2.get()

    btn_3 = tk.OptionMenu(root,
                  variable3, *OPTIONS2)
   
    choice3=variable2.get()
    btn_4 = tk.OptionMenu(root,
                  variable4, *OPTIONS2)
    choice4=variable2.get()

    btn_1.grid(row=2,column=0)
    btn_1_0.grid(row=2,column=1)
    btn_2.grid(row=3,column=0)
    btn_2_0.grid(row=3,column=1)
    btn_3.grid(row=4, column=0)
    btn_3_0.grid(row=4,column=1)
    btn_4.grid(row=5, column=0)
    btn_4_0.grid(row=5,column=1)


    
def browsefunc():
    global filename
    global entry
    filename = tk.filedialog.askopenfilename(filetypes=(("csv files","*.csv"),(("txt files","*.txt")),("All files","*.*")))
    e1.insert(tk.END,filename)
    entry = e1.get()
    

# Declaration of SourceQC input dialog
root = tk.Tk() 

tk.Label(root, text="Choose the .csv file").grid(row=1)  
e1 = tk.Entry(root) 
e1.grid(row=1, column=2)
b1=tk.Button(root,text="Browse",command=browsefunc)
b1.grid(row=1,column=3)


#Variable declaration for choices chosen by user from list
choice1='E'
choice2='E'
choice3='E'
choice4='E'

btn_0_1 = tk.Button(root,
               text = "OK",
               command = myClick0)
btn_0_1.grid(row=1,column=4)

plot_button = Button(master = root,  
                     command = plot, 
                     height = 2,  
                     width = 8, 
                     text = "Plot").grid(row=8,column=5)

tk.Label(root, text="@powered by Mapmaster",  font=("Arial Narrow", 6)).grid(row=9,column=5)   

root.title('Source QC') 
root.mainloop()




finished in %s seconds 1657619743.0685508


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\DELL\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_4672\1475179407.py", line 128, in plot
    df2.to_csv(bad_shots, index=False)
  File "C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py", line 3551, in to_csv
    return DataFrameRenderer(formatter).to_csv(
  File "C:\Users\DELL\Anaconda3\lib\site-packages\pandas\io\formats\format.py", line 1180, in to_csv
    csv_formatter.save()
  File "C:\Users\DELL\Anaconda3\lib\site-packages\pandas\io\formats\csvs.py", line 241, in save
    with get_handle(
  File "C:\Users\DELL\Anaconda3\lib\site-packages\pandas\io\common.py", line 789, in get_handle
    handle = open(
PermissionError: [Errno 13] Permission denied: 'bad_shots.csv'


clicked at 315 370
clicked at 303 393
clicked at 298 385
clicked at 273 271
clicked at 273 271
clicked at 219 432
clicked at 275 425
clicked at 292 385
clicked at 255 395
clicked at 202 405
clicked at 366 291
clicked at 366 291
clicked at 437 284
clicked at 416 282
clicked at 436 152
clicked at 155 271
clicked at 317 276
clicked at 354 240
clicked at 356 36
clicked at 320 79
